In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
url = 'https://drive.google.com/uc?id={}'.format("1WfhmjbtNr3swXnP_cHrZXukYmOPAmFCr")

In [4]:
path_to_zip = tf.keras.utils.get_file('Data.zip', origin=url, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'Data')

19484567/19484567 [==============================] - 1s 0us/step


In [23]:
batch_size = 32
img_height = 160
img_width = 160

In [24]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3884 files belonging to 2 classes.
Using 3108 files for training.


In [25]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3884 files belonging to 2 classes.
Using 776 files for validation.


In [ ]:
# class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    # plt.title(class_names[labels[i]])
    plt.axis("off")

In [26]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 5)
val_ds = val_ds.skip(val_batches // 5)

In [27]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_ds))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_ds))

Number of validation batches: 20
Number of test batches: 5


In [28]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size = AUTOTUNE)

In [29]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
for image, _ in train_ds.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

In [31]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [32]:
IMG_SHAPE = (img_width, img_height) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [33]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [34]:
base_model.trainable = False

In [35]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [36]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 1)


In [43]:
# from keras.applications.inception_v3 import preprocess_input
inputs = tf.keras.Input(shape=(160, 160, 3))
x =data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2) (x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)


In [44]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [45]:
initial_epochs = 10

In [48]:
history = model.fit(train_ds, epochs=initial_epochs, validation_data=val_ds)

Epoch 1/10
98/98 [==============================] - 48s 488ms/step - loss: 0.5985 - accuracy: 0.6660 - val_loss: 0.5970 - val_accuracy: 0.6607
Epoch 2/10
98/98 [==============================] - 51s 523ms/step - loss: 0.5836 - accuracy: 0.6644 - val_loss: 0.6056 - val_accuracy: 0.6396
Epoch 3/10
98/98 [==============================] - 56s 568ms/step - loss: 0.5950 - accuracy: 0.6596 - val_loss: 0.5947 - val_accuracy: 0.6688
Epoch 4/10
98/98 [==============================] - 54s 550ms/step - loss: 0.5702 - accuracy: 0.6924 - val_loss: 0.5827 - val_accuracy: 0.6753
Epoch 5/10
98/98 [==============================] - 52s 535ms/step - loss: 0.5661 - accuracy: 0.6969 - val_loss: 0.5924 - val_accuracy: 0.6721
Epoch 6/10
98/98 [==============================] - 54s 550ms/step - loss: 0.5694 - accuracy: 0.6892 - val_loss: 0.5814 - val_accuracy: 0.6688
Epoch 7/10
98/98 [==============================] - 53s 547ms/step - loss: 0.5680 - accuracy: 0.6992 - val_loss: 0.5852 - val_accuracy: 0.6721

In [49]:
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)

5/5 [==============================] - 2s 374ms/step - loss: 0.5339 - accuracy: 0.7000
Test accuracy : 0.699999988079071


 **Fine Tuning**

In [50]:
base_model.trainable = True

In [51]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  154


In [52]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [53]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, None, None, 3)     0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 160, 160, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 160, 160, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                           

In [54]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds)

Epoch 10/20
98/98 [==============================] - 84s 815ms/step - loss: 0.5218 - accuracy: 0.7310 - val_loss: 0.4906 - val_accuracy: 0.7354
Epoch 11/20
98/98 [==============================] - 80s 813ms/step - loss: 0.4462 - accuracy: 0.7838 - val_loss: 0.4723 - val_accuracy: 0.8084
Epoch 12/20
98/98 [==============================] - 75s 765ms/step - loss: 0.3986 - accuracy: 0.8160 - val_loss: 0.4980 - val_accuracy: 0.7045
Epoch 13/20
98/98 [==============================] - 79s 803ms/step - loss: 0.3730 - accuracy: 0.8272 - val_loss: 0.4369 - val_accuracy: 0.7630
Epoch 14/20
98/98 [==============================] - 78s 798ms/step - loss: 0.3409 - accuracy: 0.8436 - val_loss: 0.3053 - val_accuracy: 0.8653
Epoch 15/20
98/98 [==============================] - 78s 801ms/step - loss: 0.3246 - accuracy: 0.8559 - val_loss: 0.3104 - val_accuracy: 0.8442
Epoch 16/20
98/98 [==============================] - 78s 798ms/step - loss: 0.2992 - accuracy: 0.8674 - val_loss: 0.2869 - val_accuracy:

In [55]:
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)

5/5 [==============================] - 2s 358ms/step - loss: 0.2838 - accuracy: 0.8562
Test accuracy : 0.856249988079071


In [56]:
model.save("SmileDetection_model")

In [68]:
ls SmileDetection_model

assets/  fingerprint.pb  keras_metadata.pb  saved_model.pb  variables/


In [69]:
!zip -r /content/Smile_detection_model.zip /content/SmileDetection_model


  adding: content/SmileDetection_model/ (stored 0%)
  adding: content/SmileDetection_model/saved_model.pb (deflated 91%)
  adding: content/SmileDetection_model/variables/ (stored 0%)
  adding: content/SmileDetection_model/variables/variables.index (deflated 75%)
  adding: content/SmileDetection_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/SmileDetection_model/assets/ (stored 0%)
  adding: content/SmileDetection_model/keras_metadata.pb (deflated 96%)
  adding: content/SmileDetection_model/fingerprint.pb (stored 0%)


In [70]:
from google.colab import files
files.download('Smile_detection_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>